# Building ADML From Scratch

In the last section we saw how ADML works. We saw how we train our model with both clean and adversarial samples to obtain a better and robust model parameter $\theta$ that is generalizabel across tasks. 


Now we will better understand ADML by coding them from scratch. For better understanding, we consider a simple binary classification task. We randomly generate our input data and we train them with a single layer neural network and try to find the optimal parameter $\theta$. 

Now we will step by step how exacly we are doing this,

First we import all the necessary libraries,

In [1]:
import numpy as np
import tensorflow as tf

## Generate Clean Samples

Now we define a function called sample_points for generating clean input (x,y) pairs. It takes the parameter k as an input which implies number of (x,y) pairs we want to sample. 

In [2]:
def sample_points(k):
    x = np.random.rand(k,50)
    y = np.random.choice([0, 1], size=k, p=[.5, .5]).reshape([-1,1])
    return x,y

The above function returns output as follows, 

In [3]:
x, y = sample_points(10)
print x[0]
print y[0]

[0.88181038 0.51678418 0.81979321 0.32583704 0.12557307 0.30478364
 0.96272806 0.00636423 0.0375513  0.05887383 0.52036732 0.66574474
 0.45427606 0.28874493 0.19352738 0.44417903 0.63303766 0.93938007
 0.05098809 0.10348272 0.1037544  0.59786084 0.64339621 0.07255698
 0.3412259  0.31813905 0.20525812 0.20301659 0.90735178 0.14418036
 0.57155755 0.53092418 0.62812951 0.96301036 0.75604885 0.01451255
 0.21633051 0.34509995 0.57552133 0.82287969 0.7935126  0.50339959
 0.62920624 0.6642846  0.41366068 0.12444433 0.09815865 0.15425461
 0.43131473 0.64740023]
[0]


## Generate Adversarial Samples

Now, we define one more function called FGSM for generating adversarial inputs. We use Fast Gradient Sign Method (FGSM) for generating adversarial samples. We have seen how FGSM generates the adversarial pairs by calculating gradients with respect to the inputs instead of model parameter. So, We take clean (x,y) pairs as an input and generate adversarial (x_adv,y) pairs.

In [4]:
def FGSM(x,y):

    #placeholder for the inputs x and y
    X = tf.placeholder(tf.float32)
    Y = tf.placeholder(tf.float32)

    #initialize theta with random values
    theta = tf.Variable(tf.zeros([50,1]))

    #predict the value of y
    YHat = tf.nn.softmax(tf.matmul(X, theta)) 

    #calculate the loss
    loss = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(YHat), reduction_indices=1))
 
    #now calculate gradient of our loss function with respect to our input X instead of model parameter theta
    gradient = ((tf.gradients(loss,X)[0]))
    
    #calculate the adversarial input
    #i.e x_adv = x + epsilon * sign ( nabla_x J(X, Y))
    X_adv = X + 0.2*tf.sign(gradient)
    X_adv = tf.clip_by_value(X_adv,-1.0,1.0)

    #start the tensoflow session
    with tf.Session() as sess:

        sess.run(tf.global_variables_initializer())        
        X_adv = sess.run(X_adv, feed_dict={X: x, Y: y})
        
    return X_adv, y

In [5]:
X_adv , Y = FGSM(x,y)

## Single Layer Neural Network

We use a neural network with a single layer for predicting the output.

a = np.matmul(X, theta)


YHat = sigmoid(a)

So, we use ADML for finding this optimal parameter value theta that is generalizable across tasks. So for a new task, we can learn from a few data points in a lesser time by taking very less gradient steps.

## Adversarial Meta Learning (ADML)

Now, we define a class called ADML where we implement the ADML algorithm. In the \__init__  method we will initialize all the necessary variables. Then we define our sigmoid function. Followed by we define our train function. 



You can check the comments written above each line of code for understanding.

In [6]:
class ADML(object):
    def __init__(self):

        #initialize number of tasks i.e number of tasks we need in each batch of tasks
        self.num_tasks = 2
        
        #number of samples i.e number of shots  -number of data points (k) we need to have in each task
        self.num_samples = 10

        #number of epochs i.e training iterations
        self.epochs = 100
        
        #hyperparameter for the inner loop (inner gradient update)

        #for clean sample
        self.alpha1 = 0.0001

        #for adversarial sample
        self.alpha2 = 0.0001
        
        #hyperparameter for the outer loop (outer gradient update) i.e meta optimization
        
        #for clean sample
        self.beta1 = 0.0001
        
        #for adversarial sample
        self.beta2 = 0.0001

        #randomly initialize our model parameter theta
        self.theta = np.random.normal(size=50).reshape(50, 1)

    #define our sigmoid activation function  
    def sigmoid(self,a):
        return 1.0 / (1 + np.exp(-a))
    
    #now let us get to the interesting part i.e training :P
    def train(self):
        
        #for the number of epochs,
        for e in range(self.epochs):        
            
            #theta' of clean samples
            self.theta_clean = []

            #theta' of adversarial samples
            self.theta_adv = []
            
            #for task i in batch of tasks
            for i in range(self.num_tasks):
                
                #sample k data points and prepare our training data

                #first, we sample clean data points
                XTrain_clean, YTrain_clean = sample_points(self.num_samples)

                #feed the clean samples to FGSM and get adversarial samples
                XTrain_adv, YTrain_adv = FGSM(XTrain_clean,YTrain_clean)

                #1. First, we computer theta' for clean samples and store it in theta_clean
                
                #predict the output y 
                a = np.matmul(XTrain_clean, self.theta)

                YHat = self.sigmoid(a)

                #since we are performing classification, we use cross entropy loss as our loss function
                loss = ((np.matmul(-YTrain_clean.T, np.log(YHat)) - np.matmul((1 -YTrain_clean.T), np.log(1 - YHat)))/self.num_samples)[0][0]
                
                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain_clean.T, (YHat - YTrain_clean)) / self.num_samples

                #update the gradients and find the optimal parameter theta' for clean samples
                self.theta_clean.append(self.theta - self.alpha1*gradient)
              
                #2. Now, we compute theta' for adversarial samples and store it in theta_clean

                #predict the output y 
                a = (np.matmul(XTrain_adv, self.theta))

                YHat = self.sigmoid(a)

                #calculate cross entropy loss
                loss = ((np.matmul(-YTrain_adv.T, np.log(YHat)) - np.matmul((1 -YTrain_adv.T), np.log(1 - YHat)))/self.num_samples)[0][0]
                
                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain_adv.T, (YHat - YTrain_adv)) / self.num_samples

                #update the gradients and find the optimal parameter theta' for adversarial samples
                self.theta_adv.append(self.theta - self.alpha2*gradient)
                
            #initialize meta gradients for clean samples
            meta_gradient_clean = np.zeros(self.theta.shape)

            #initialize meta gradients for adversarial samples
            meta_gradient_adv = np.zeros(self.theta.shape)
            
            for i in range(self.num_tasks):
                
                #sample k data points and prepare our test set for meta training

                #first, we sample clean data points
                XTest_clean, YTest_clean = sample_points(self.num_samples)

                #feed the clean samples to FGSM and get adversarial samples
                XTest_adv, YTest_adv = FGSM(XTest_clean,YTest_clean)
                       
                #1. First, we computer meta gradients for clean samples 

                #predict the value of y
                a = np.matmul(XTest_clean, self.theta_clean[i])
                
                YPred = self.sigmoid(a)
                           
                #compute meta gradients
                meta_gradient_clean += np.matmul(XTest_clean.T, (YPred - YTest_clean)) / self.num_samples

                #2. Now, we compute meta gradients for adversarial samples
                
                #predict the value of y
                a = (np.matmul(XTest_adv, self.theta_adv[i]))
                
                YPred = self.sigmoid(a)
                           
                #compute meta gradients
                meta_gradient_adv += np.matmul(XTest_adv.T, (YPred - YTest_adv)) / self.num_samples

            #update our randomly initialized model parameter theta
            #with the meta gradients of both clean and adversarial samples
            
            self.theta = self.theta-self.beta1*meta_gradient_clean/self.num_tasks

            self.theta = self.theta-self.beta2*meta_gradient_adv/self.num_tasks
                                  
            if e%10==0:
                print "Epoch {}: Loss {}\n".format(e,loss)             
                print 'Updated Model Parameter Theta\n'
                print 'Sampling Next Batch of Tasks \n'
                print '---------------------------------\n'

In [7]:
model = ADML()

In [ ]:
model.train()

Epoch 0: Loss 1.06869670787

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 10: Loss 2.20820318343

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 20: Loss 0.556802980902

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

Epoch 30: Loss 1.16247761064

Updated Model Parameter Theta

Sampling Next Batch of Tasks 

---------------------------------

